In [22]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# import lightgbm
from lightgbm import LGBMRegressor
import datetime

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
rcParams['figure.figsize'] = (16, 8)
rc('font', family='AppleGothic')

In [23]:
train_path = './trainingdata'
infer_path = './inferencedata'
submission_path = './submission'
importance_path = './feature_importance'
oof_path = './oof_preds'
pil_path = './magok'

In [24]:
submission = pd.read_csv('./data/sample_submission.csv')
submission2 = pd.read_csv('./data/sample_submission.csv')

In [4]:
# 테스트 time_str
time_str = '20210105_221155_변수소거테스트'

In [25]:
time_str = '20210117_133024'
train = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}.csv')
)
X_test = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}.csv')
)

In [26]:
train_day23 = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}_day23.csv')
)
X_test_day23 = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}_day23.csv')
)

In [27]:
train['Hour'] = train['Hour'].astype('category')
train_day23['Hour'] = train_day23['Hour'].astype('category')
X_test['Hour'] = X_test['Hour'].astype('category')
X_test_day23['Hour'] = X_test_day23['Hour'].astype('category')

# Metric

In [28]:
def pinball(pred, actual):
    quantile_col = pred.columns.tolist()
    pred2 = pred.copy()
    actual2 = actual.copy()
    
    pred2.index = range(len(pred2))
    actual2.index = range(len(actual2))

    pinball_loss = []
    for item in quantile_col:
        pre_pinball = []
        for idx in range(len(pred2)):
            y = actual2.iloc[idx]
            z = pred2[item].iloc[idx]
            q = item
            
            if y >= z:
                value = (y - z)*q
                pre_pinball.append(value)
            else:
                value = (z - y)*(1 - q)
                pre_pinball.append(value)
        loss_mean = np.mean(pre_pinball)
        pinball_loss.append(loss_mean)
    
    fin = np.mean(pinball_loss)
    print(f'Pinball Loss: {fin}')
    
    return pinball_loss         

# Inference

In [29]:
# X_train = X_train.drop(columns = drop_col)
# X_test = X_test.drop(columns = drop_col)

In [30]:
X_train = train.drop(columns = ['1day_after_target', '2day_after_target'])
target_1 = train['1day_after_target']
target_2 = train['2day_after_target']

In [31]:
X_train_day23 = train_day23.drop(columns = ['1day_after_target', '2day_after_target'])
target_1_day23 = train_day23['1day_after_target']
target_2_day23 = train_day23['2day_after_target']

In [33]:
def LGBM_reg(X_train,y_train, test, target_num, seed):
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    # 랜덤모수 샘플 추론이 왜 안되지;;;
    # fold = StratifiedKFold(n_splits=5, shuffle = True)
    
    fold = KFold(n_splits=7, shuffle = True, random_state = seed)

    oof_preds = np.zeros([X_train.shape[0], len(quantiles)])
    
    feature_importance_df = pd.DataFrame(np.zeros([len(X_train.columns), len(quantiles)]))
    feature_importance_df.columns = quantiles
    
    test_df = pd.DataFrame(np.zeros([test.shape[0], len(quantiles)]))
    test_df.columns = quantiles
    
    fold_metric = []
    # training
    target = y_train
    for fold_, (train_idx, valid_idx) in enumerate(
        fold.split(np.array(X_train), target)
    ):
        
#         if fold_ >0:
#             break
        train_x, train_y = X_train.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = X_train.iloc[valid_idx], target.iloc[valid_idx]

        oof_df = pd.DataFrame()
        for idx, q in enumerate(quantiles):
            print(f'\nquantile: {q}\n')
            model = LGBMRegressor(objective='quantile',
                                  alpha=q,
                                  n_estimators=10000, 
                                  bagging_fraction=0.7, 
                                  learning_rate=0.027, 
                                  subsample=0.7
                                 )   

            model.fit(train_x, train_y, eval_metric = ['quantile'], 
              eval_set=[(valid_x, valid_y)], early_stopping_rounds=300, verbose=1500)

            oof_preds[[valid_idx],idx] = \
                model.predict(valid_x).round(2)

            oof_df[q] = model.predict(valid_x).round(2)
            test_df[q] += model.predict(test).round(2) / fold.n_splits
            
            feature_importance_df[q] = model.feature_importances_ / 7
            
        metric = pinball(oof_df, valid_y)
        loss = np.mean(metric)
        print(f'\n FOLD {fold_}의 total pinball loss: {loss}\n')
        fold_metric.append(metric)
        # feature importance
        
        print('\n===================================================')
        print(f'FOLD {fold_} Success')
        print('===================================================\n')
        
    feature_importance_df.index = X_train.columns.tolist()
    feature_importance_df2 = feature_importance_df.reset_index()
    
    feature_importance_df2.to_csv(f'{importance_path}/{time_str}_{target_num}_fe.csv', index = False)
    pd.DataFrame(oof_preds).to_csv(f'{oof_path}/{time_str}_{target_num}_oof_pred.csv', index = False)
    
    return oof_preds, fold_metric, test_df, feature_importance_df2

In [34]:
oof_preds, fold_metric, test_df, feature_imp = \
    LGBM_reg(X_train, target_1, X_test,1, 1001)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[620]	valid_0's quantile: 1.34611

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.03335
[3000]	valid_0's quantile: 2.01981
Early stopping, best iteration is:
[3423]	valid_0's quantile: 2.01833

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.39855
[3000]	valid_0's quantile: 2.35169
[4500]	valid_0's quantile: 2.32266
Early stopping, best iteration is:
[4654]	valid_0's quantile: 2.32107

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.509
[3000]	valid_0's quantile: 2.46124
[4500]	valid_0's quantile: 2.4189
[6000]	valid_0's quantile: 2.40201
[7500]	valid_0's quantile: 2.38906
[9000]	valid_0's quantile: 2.38048
Did not meet early stopping. Best iteration is:
[9999]	valid_0's quantile: 2.37601

quantile: 0.5


quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.45836
[3000]	valid_0's quantile: 2.42072
Early stopping, best iteration is:
[2818]	valid_0's quantile: 2.41926

quantile: 0.6

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.18931
[3000]	valid_0's quantile: 2.17022
[4500]	valid_0's quantile: 2.15362
[6000]	valid_0's quantile: 2.14485
[7500]	valid_0's quantile: 2.13508
[9000]	valid_0's quantile: 2.13134
Early stopping, best iteration is:
[9338]	valid_0's quantile: 2.13039

quantile: 0.7

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.77376
Early stopping, best iteration is:
[2616]	valid_0's quantile: 1.7628

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.2802
Early stopping, best iteration is:
[1752]	valid_0's quantile: 1.27672

quantile: 0.9

Training until validation scores don't impro

In [35]:
oof_preds2, fold_metric2, test_df2, feature_imp2 = \
    LGBM_reg(X_train, target_2, X_test,2, 1001)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[667]	valid_0's quantile: 1.39667

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.07069
Early stopping, best iteration is:
[1981]	valid_0's quantile: 2.06122

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.4004
[3000]	valid_0's quantile: 2.3631
[4500]	valid_0's quantile: 2.33863
[6000]	valid_0's quantile: 2.32099
Early stopping, best iteration is:
[5777]	valid_0's quantile: 2.32077

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.52493
[3000]	valid_0's quantile: 2.47338
[4500]	valid_0's quantile: 2.4505
[6000]	valid_0's quantile: 2.42973
[7500]	valid_0's quantile: 2.42034
[9000]	valid_0's quantile: 2.41479
Early stopping, best iteration is:
[9513]	valid_0's quantile: 2.41328

quantile: 0.5

Training un

[7500]	valid_0's quantile: 2.47931
Early stopping, best iteration is:
[8227]	valid_0's quantile: 2.47488

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.49027
Early stopping, best iteration is:
[2126]	valid_0's quantile: 2.46111

quantile: 0.6

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.24226
[3000]	valid_0's quantile: 2.21867
[4500]	valid_0's quantile: 2.20985
Early stopping, best iteration is:
[5389]	valid_0's quantile: 2.20422

quantile: 0.7

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.82962
Early stopping, best iteration is:
[2583]	valid_0's quantile: 1.81809

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.30873
[3000]	valid_0's quantile: 1.30068
Early stopping, best iteration is:
[3439]	valid_0's quantile: 1.29973

quantile: 0.9

Training until validation scores don't imp


quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.38606
Early stopping, best iteration is:
[2542]	valid_0's quantile: 1.37784

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.750027
[3000]	valid_0's quantile: 0.747427
Early stopping, best iteration is:
[3062]	valid_0's quantile: 0.747351
Pinball Loss: 1.916725200802727

 FOLD 6의 total pinball loss: 1.916725200802727


FOLD 6 Success



In [36]:
oof_preds_day23, fold_metric_day23, test_df_day23, feature_imp_day23 = \
    LGBM_reg(X_train_day23, target_1_day23, X_test_day23,11, 1001)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[731]	valid_0's quantile: 1.35768

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.06017
[3000]	valid_0's quantile: 2.03236
[4500]	valid_0's quantile: 2.02214
Early stopping, best iteration is:
[4541]	valid_0's quantile: 2.02148

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.41978
[3000]	valid_0's quantile: 2.36667
[4500]	valid_0's quantile: 2.34472
[6000]	valid_0's quantile: 2.33609
[7500]	valid_0's quantile: 2.32168
[9000]	valid_0's quantile: 2.31585
Early stopping, best iteration is:
[9642]	valid_0's quantile: 2.31447

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.53226
[3000]	valid_0's quantile: 2.48329
[4500]	valid_0's quantile: 2.45514
[6000]	valid_0's quantile: 2.42893
[7500]	valid_0's quantil

[1500]	valid_0's quantile: 2.47996
[3000]	valid_0's quantile: 2.4232
[4500]	valid_0's quantile: 2.3978
[6000]	valid_0's quantile: 2.38381
[7500]	valid_0's quantile: 2.37649
[9000]	valid_0's quantile: 2.36493
Did not meet early stopping. Best iteration is:
[10000]	valid_0's quantile: 2.3618

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.54279
[3000]	valid_0's quantile: 2.49213
[4500]	valid_0's quantile: 2.4567
[6000]	valid_0's quantile: 2.44007
[7500]	valid_0's quantile: 2.42895
[9000]	valid_0's quantile: 2.42134
Did not meet early stopping. Best iteration is:
[10000]	valid_0's quantile: 2.41722

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.47622
[3000]	valid_0's quantile: 2.42756
[4500]	valid_0's quantile: 2.41245
[6000]	valid_0's quantile: 2.39991
[7500]	valid_0's quantile: 2.39081
[9000]	valid_0's quantile: 2.38493
Did not meet early stopping. Best iteration is:
[9

[1500]	valid_0's quantile: 2.33104
[3000]	valid_0's quantile: 2.29371
[4500]	valid_0's quantile: 2.27917
[6000]	valid_0's quantile: 2.26378
[7500]	valid_0's quantile: 2.25528
[9000]	valid_0's quantile: 2.25018
Did not meet early stopping. Best iteration is:
[9972]	valid_0's quantile: 2.2476

quantile: 0.7

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.93634
[3000]	valid_0's quantile: 1.91674
Early stopping, best iteration is:
[3672]	valid_0's quantile: 1.9102

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.40008
[3000]	valid_0's quantile: 1.39298
Early stopping, best iteration is:
[3636]	valid_0's quantile: 1.39052

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.759675
[3000]	valid_0's quantile: 0.757882
Early stopping, best iteration is:
[4049]	valid_0's quantile: 0.75717
Pinball Loss: 1.9240360157327336

 FOLD 6의 total pinb

In [37]:
oof_preds2_day23, fold_metric2_day23, test_df2_day23, feature_imp2_day23 = \
    LGBM_reg(X_train_day23, target_2_day23, X_test_day23,22, 1001)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.35456
Early stopping, best iteration is:
[1419]	valid_0's quantile: 1.35412

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.10857
[3000]	valid_0's quantile: 2.08515
Early stopping, best iteration is:
[3175]	valid_0's quantile: 2.08224

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.46851
[3000]	valid_0's quantile: 2.41686
[4500]	valid_0's quantile: 2.40522
[6000]	valid_0's quantile: 2.39832
Early stopping, best iteration is:
[6833]	valid_0's quantile: 2.39525

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.61843
[3000]	valid_0's quantile: 2.55267
[4500]	valid_0's quantile: 2.53293
[6000]	valid_0's quantile: 2.51775
Early stopping, best iteration is:
[6786]	valid_0's quantile: 2.50793

quantile: 0.5

Trainin


quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.48029
[3000]	valid_0's quantile: 2.42364
[4500]	valid_0's quantile: 2.39628
[6000]	valid_0's quantile: 2.37979
[7500]	valid_0's quantile: 2.37119
[9000]	valid_0's quantile: 2.36698
Did not meet early stopping. Best iteration is:
[10000]	valid_0's quantile: 2.36459

quantile: 0.6

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.22557
[3000]	valid_0's quantile: 2.17945
[4500]	valid_0's quantile: 2.16133
Early stopping, best iteration is:
[4523]	valid_0's quantile: 2.16097

quantile: 0.7

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.81702
[3000]	valid_0's quantile: 1.79852
Early stopping, best iteration is:
[3000]	valid_0's quantile: 1.79852

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.31815
[3000]	valid_0's quantile: 1.30646
[4500]	va


quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.35034
[3000]	valid_0's quantile: 1.33526
[4500]	valid_0's quantile: 1.33289
Early stopping, best iteration is:
[4201]	valid_0's quantile: 1.33177

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.738947
[3000]	valid_0's quantile: 0.736997
Early stopping, best iteration is:
[3025]	valid_0's quantile: 0.736986
Pinball Loss: 1.8867344754959963

 FOLD 6의 total pinball loss: 1.8867344754959963


FOLD 6 Success



# submission

In [39]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = test_df.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = test_df2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
submission2.loc[submission2.id.str.contains("Day7"), "q_0.1":] = test_df_day23.sort_index().values
submission2.loc[submission2.id.str.contains("Day8"), "q_0.1":] = test_df2_day23.sort_index().values
submission2

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Ensemble

In [40]:
submission3 = submission[['id']]
sub_value = submission.iloc[:,1:] * 0.7 + submission2.iloc[:,1:] * 0.3

submission3 = pd.concat([submission3,sub_value],axis = 1)

# 제출

In [63]:
# time_str = datetime.datetime.strftime(
#     datetime.datetime.today(),
#     '%Y%m%d_%H%M%S'
# )

# training 데이터 저장한 시간이랑 맞춰서 사용
submission.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}.csv'), 
    index=False
)

In [41]:
# 앙상블 결과 제출
submission3.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}_ensemble_가중평균.csv'), 
    index=False
)

- 가중평균: 0.7/0.3
- 가중평균2: 0.8/0.2
- 가중평균3: 0.6/0.4

# 외부 데이터와 앙상블(magok)

In [42]:
# 필령이형 파일 앙상블
time_str = '20210115_140714'
time_str2 = '20210117_133024'
temp = pd.read_csv(os.path.join(submission_path, f'제출용_{time_str}_ensemble_가중평균.csv'))
target = pd.read_csv(os.path.join(submission_path, f'제출용_{time_str2}_ensemble_가중평균.csv'))

target.iloc[:,1:] = temp.iloc[:,1:] * 0.5 + target.iloc[:,1:] * 0.5

In [43]:
# time_str = datetime.datetime.strftime(
#     datetime.datetime.today(),
#     '%Y%m%d_%H%M%S'
# )

target.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}_{time_str2}_ensemble_external_submission_ensemble_v2_weight05.csv'), 
    index=False
)

# 변수중요도 데이터 추출

In [46]:
time_str = '20210105_221155'
imp_path1 = f'{importance_path}/{time_str}_1_fe.csv'
imp_path2 = f'{importance_path}/{time_str}_2_fe.csv'

imp1 = pd.read_csv(
    imp_path1
)
imp2 = pd.read_csv(
    imp_path2
)

In [48]:
drop_col = ['DNI_label','TARGET_interval','2days_mean_TARGET_rolling']

# inference data load

In [216]:
infer_time = '20210107_231340'
temp = pd.read_csv(
    os.path.join(submission_path,f'제출용_{infer_time}_ensemble_가중평균.csv')
)    

In [219]:
submission3[submission3['q_0.2'] > 0]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
1464,15.csv_Day7_12h00m,0.0583,0.0317,0.4843,0.2177,0.1127,-0.3424,-0.5417,-0.1243,0.3677
1465,15.csv_Day7_12h30m,0.0000,0.1504,0.3266,0.1161,0.1414,-0.3514,-0.3390,0.0201,0.3806
1483,15.csv_Day7_21h30m,0.0000,0.0947,0.3749,-0.1961,-0.6714,-0.7457,-0.7140,0.0957,0.6906
1487,15.csv_Day7_23h30m,0.5910,0.1130,-0.8010,-0.2110,-0.9950,-1.4290,-1.4220,1.2080,1.4630
1512,15.csv_Day8_12h00m,0.6081,0.9960,0.5841,0.4749,0.3951,0.3527,0.1316,0.3450,0.3909
...,...,...,...,...,...,...,...,...,...,...
6455,67.csv_Day7_11h30m,0.0000,0.0440,0.3720,-0.2900,-0.3560,-0.4870,-0.9970,0.2280,0.5550
6480,67.csv_Day8_0h00m,0.1240,0.1250,0.1100,0.1360,0.2830,0.3890,0.3810,0.2710,0.4630
6481,67.csv_Day8_0h30m,-0.0220,0.0430,-0.2070,-0.3850,-1.1430,-1.4040,-0.8530,0.1530,0.1000
6502,67.csv_Day8_11h00m,0.6560,0.2460,0.8870,1.0260,0.6070,-1.0190,0.5090,0.9180,1.2640


# CV성능 검증

In [172]:
time_str = '20210115_140714'
target_num = 1

oof_df = pd.read_csv(f'{oof_path}/{time_str}_{target_num}_oof_pred.csv')

train_oof = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}.csv')
)

quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
oof_df.columns = quantiles

In [173]:
pinball(oof_df, train_oof['1day_after_target'])

Pinball Loss: 1.869713283841473


[1.3695450439544232,
 2.0806082713459033,
 2.4221420761109322,
 2.5210915980336783,
 2.4169633887550255,
 2.2015429073153294,
 1.8036546910089106,
 1.298799747233424,
 0.7130718308156289]